In [7]:

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

import folium # map rendering library

from geopy.geocoders import Nominatim
from geopy import distance
from pygeocoder import Geocoder

import pickle

print('Libraries imported.')

Libraries imported.


In [8]:
sf_geospatial_data_zip = pd.read_pickle("data/sf_geospatial_data_zip.pkl")
sf_geospatial_data = pd.read_pickle("data/sf_geospatial_data.pkl")
categories = pd.read_pickle("data/categories.pkl")
venues = pd.read_pickle("data/venues.pkl")


In [7]:
print("No. of Neighbouhoods: {}".format(sf_geospatial_data.shape[0]))
print("No. of Postal Codes: {}".format(sf_geospatial_data_zip.shape[0]))
print("No. of Categories: {}".format(categories.shape[0]))
print("No. of Venues: {}".format(venues.shape[0]))

No. of Neighbouhoods: 116
No. of Postal Codes: 27
No. of Categories: 937
No. of Venues: 5549


In [468]:
sf_geospatial_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 116 entries, 0 to 116
Data columns (total 3 columns):
Neighbourhood    116 non-null object
latitude         116 non-null float64
longitude        116 non-null float64
dtypes: float64(2), object(1)
memory usage: 3.6+ KB


In [469]:
sf_geospatial_data.head()

,Neighbourhood,latitude,longitude
0,Seacliff,37.785941,-122.490700
1,Lake Street,37.786197,-122.475521
2,Presidio National Park,37.807044,-122.472101
3,Presidio Terrace,37.788263,-122.460805
4,Inner Richmond,37.779870,-122.464746


In [470]:
sf_geospatial_data_zip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 3 columns):
postal_code    27 non-null object
latitude       27 non-null float64
longitude      27 non-null float64
dtypes: float64(2), object(1)
memory usage: 728.0+ bytes


In [471]:
sf_geospatial_data_zip.head()

,postal_code,latitude,longitude
0,94124,37.730417,-122.384425
1,94127,37.734646,-122.463708
2,94131,37.740104,-122.438231
3,94133,37.805989,-122.409915
4,94132,37.718140,-122.486349


In [472]:
categories.info()

<class 'pandas.core.frame.DataFrame'>
Index: 937 entries, 4d4b7104d754a06370d81259 to 52f2ab2ebcbc57f1066b8b4a
Data columns (total 3 columns):
name         937 non-null object
icon         937 non-null object
parent_id    927 non-null object
dtypes: object(3)
memory usage: 29.3+ KB


In [473]:
categories.head()

,name,icon,parent_id
4d4b7104d754a06370d81259,Arts & Entertainment,https://ss3.4sqi.net/img/categories_v2/arts_en...,NaN
56aa371be4b08b9a8d5734db,Amphitheater,https://ss3.4sqi.net/img/categories_v2/arts_en...,4d4b7104d754a06370d81259
4fceea171983d5d06c3e9823,Aquarium,https://ss3.4sqi.net/img/categories_v2/arts_en...,4d4b7104d754a06370d81259
4bf58dd8d48988d1e1931735,Arcade,https://ss3.4sqi.net/img/categories_v2/arts_en...,4d4b7104d754a06370d81259
4bf58dd8d48988d1e2931735,Art Gallery,https://ss3.4sqi.net/img/categories_v2/arts_en...,4d4b7104d754a06370d81259


In [474]:
categories[pd.isnull(categories['parent_id'])]

,name,icon,parent_id
4d4b7104d754a06370d81259,Arts & Entertainment,https://ss3.4sqi.net/img/categories_v2/arts_en...,NaN
4d4b7105d754a06372d81259,College & University,https://ss3.4sqi.net/img/categories_v2/educati...,None
4d4b7105d754a06373d81259,Event,https://ss3.4sqi.net/img/categories_v2/event/d...,None
4d4b7105d754a06374d81259,Food,https://ss3.4sqi.net/img/categories_v2/food/de...,None
4d4b7105d754a06376d81259,Nightlife Spot,https://ss3.4sqi.net/img/categories_v2/nightli...,None
4d4b7105d754a06377d81259,Outdoors & Recreation,https://ss3.4sqi.net/img/categories_v2/parks_o...,None
4d4b7105d754a06375d81259,Professional & Other Places,https://ss3.4sqi.net/img/categories_v2/buildin...,None
4e67e38e036454776db1fb3a,Residence,https://ss3.4sqi.net/img/categories_v2/buildin...,None
4d4b7105d754a06378d81259,Shop & Service,https://ss3.4sqi.net/img/categories_v2/shops/d...,None
4d4b7105d754a06379d81259,Travel & Transport,https://ss3.4sqi.net/img/categories_v2/travel/...,None


In [475]:
venues.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5549 entries, 4f3709abe4b0aa62a1d35520 to 528c2945498ed98d2451a535
Data columns (total 10 columns):
address         5263 non-null object
category        5549 non-null object
city            5536 non-null object
country         5549 non-null object
cross_street    4107 non-null object
lat             5549 non-null float64
lng             5549 non-null float64
name            5549 non-null object
postal_code     5266 non-null object
state           5549 non-null object
dtypes: float64(2), object(8)
memory usage: 476.9+ KB


In [476]:
venues.head()

,address,category,city,country,cross_street,lat,lng,name,postal_code,state
venue_id,,,,,,,,,,
4f3709abe4b0aa62a1d35520,748 Innes Ave,4bf58dd8d48988d1ed941735,San Francisco,United States,NaN,37.730525,-122.372998,Archimedes Banya,94124,CA
49d3e08bf964a5201f5c1fe3,1634 Jerrold Ave,4bf58dd8d48988d11b951735,San Francisco,United States,NaN,37.739589,-122.390171,Flora Grubb Gardens,94124,CA
53058fdb498e2837542f7f8f,1439 Egbert Ave Unit c,4e0e22f5a56208c4ea9a85a0,San Francisco,United States,3rd Street,37.723004,-122.393615,Seven Stills Brewery & Distillery,94124,CA
555e58ad498e9137bc49f191,900 Marin St,4bf58dd8d48988d1e5931735,San Francisco,United States,btwn Illinois & Michigan St,37.749404,-122.386016,The Midway SF,94124,CA
56eb4e0a498e5769545c1843,1525 Cortland Ave,50327c8591d4c4b30a586d5d,San Francisco,United States,NaN,37.739798,-122.408986,Barebottle Brewing Company,NaN,CA


In [477]:
venues[pd.isnull(venues['postal_code'])].shape

(283, 10)

In [478]:
venues[pd.isnull(venues['postal_code'])].head()

,address,category,city,country,cross_street,lat,lng,name,postal_code,state
venue_id,,,,,,,,,,
56eb4e0a498e5769545c1843,1525 Cortland Ave,50327c8591d4c4b30a586d5d,San Francisco,United States,NaN,37.739798,-122.408986,Barebottle Brewing Company,NaN,CA
4a2af3b9f964a52073961fe3,23rd,4bf58dd8d48988d1cb941735,San Francisco,United States,Treat,37.754239,-122.413010,El Gallo Giro (Taco Truck),NaN,CA
56195ac6498e228012dfd22d,4102 24th St,4bf58dd8d48988d114951735,San Francisco,United States,Castro,37.751357,-122.434281,Charlie's Corner,NaN,CA
562405f6498e32f69e70a7a3,NaN,4bf58dd8d48988d1cb941735,San Francisco,United States,NaN,37.764285,-122.421842,Señor Sisig,NaN,CA
56e36821498efa4a86f73408,549 Irving St,5bae9231bedf3950379f89d4,San Francisco,United States,at 7th Ave,37.764130,-122.463995,Poki Time,NaN,CA


In [479]:
geolocator = Nominatim(user_agent="sf_explorer")
location = geolocator.reverse("37.739798, -122.408986")
location

Location(Barebottle Brewing, 1525, Cortland Avenue, Bernal Heights, SF, California, CA 94110, USA, (37.74000915, -122.40904048683, 0.0))

In [482]:
def get_postal_code(lat, lng, postal_code_list):
    geolocator = Nominatim(user_agent="sf_explorer")
    location = geolocator.reverse("{}, {}".format(lat, lng))
    #print(location)
    addr = location.address
    for postal_code in postal_code_list:
        if postal_code in addr:
            return postal_code
    return np.NaN

In [481]:
get_postal_code(37.739798, -122.408986, sf_geospatial_data_zip['postal_code'])

Barebottle Brewing, 1525, Cortland Avenue, Bernal Heights, SF, California, CA 94110, USA


'94110'

In [484]:
postal_code_list = sf_geospatial_data_zip['postal_code']
for index, row in venues[pd.isnull(venues['postal_code'])].iterrows():
    if pd.isnull(row['postal_code']):
        lat = row['lat']
        lng = row['lng']
        venues.set_value(index, 'postal_code', get_postal_code(lat, lng, postal_code_list))

In [485]:
venues[pd.isnull(venues['postal_code'])].shape

(54, 10)

In [486]:
venues[pd.isnull(venues['postal_code'])].head()

,address,category,city,country,cross_street,lat,lng,name,postal_code,state
venue_id,,,,,,,,,,
4d7009c9516b8cfae0615b10,SFMOMA,4bf58dd8d48988d18f941735,San Francisco,United States,NaN,37.786336,-122.400404,SFMOMA Rooftop Sculpture Garden,NaN,CA
4bda49dc2a3a0f4797b3aab6,1 Yacht Road,4bf58dd8d48988d163941735,San Francisco,United States,NaN,37.805478,-122.447863,Little Marina Green,NaN,CA
4e5425191838d8d1daccc285,San Francisco Zoo,4bf58dd8d48988d1e2941735,San Francisco,United States,NaN,37.734997,-122.507198,Pelican Beach,NaN,CA
4e54263dd4c0fe0342769ac7,San Francisco Zoo,58daa1558bbb0b01f18ec1fd,San Francisco,United States,NaN,37.734900,-122.502314,Austrailian Walkabout,NaN,CA
4dd0145a8877851244203092,San Francisco Zoo,58daa1558bbb0b01f18ec1fd,San Francisco,United States,NaN,37.732889,-122.504547,African Savana,NaN,CA


In [487]:
location = geolocator.reverse("37.805478, -122.447863")
location

Location(Mason Street, Marina District, SF, California, 14123, USA, (37.8051231020189, -122.447838576913, 0.0))

In [488]:
GOOGLE_API_KEY = 'API_KEY'
google_geocoder = Geocoder(GOOGLE_API_KEY)
result = google_geocoder.reverse_geocode(37.805478, -122.447863)
print(result)

Little Marina Green Picnic Area, 3950 Scott St, San Francisco, CA 94123, USA


In [489]:
def get_postal_code(api_key, lat, lng):
    google_geocoder = Geocoder(api_key)
    result = google_geocoder.reverse_geocode(lat, lng)
    return result.postal_code

In [490]:
postal_code_list = sf_geospatial_data_zip['postal_code']
for index, row in venues[pd.isnull(venues['postal_code'])].iterrows():
    if pd.isnull(row['postal_code']):
        lat = row['lat']
        lng = row['lng']
        venues.set_value(index, 'postal_code', get_postal_code(GOOGLE_API_KEY, lat, lng))

In [491]:
venues[pd.isnull(venues['postal_code'])].shape

(0, 10)

In [492]:
venues['postal_code'].value_counts()

94103         461
94109         409
94110         394
94102         376
94133         335
94117         296
94114         290
94115         269
94123         269
94107         251
94118         223
94122         207
94111         201
94132         189
94105         180
94108         177
94121         176
94112         155
94127         110
94116         100
94129          90
94131          76
94124          70
94104          68
94158          61
94134          54
94130          18
94143           9
94015           8
94014           6
94941           2
94199           2
94608           2
94612           1
94107-2119      1
94113           1
94107-2115      1
94111-4209      1
94080           1
94101           1
94621           1
94607           1
94619           1
94578-1972      1
94965           1
94401           1
94115-2221      1
94920           1
Name: postal_code, dtype: int64

In [493]:
venues['postal_code'] = venues['postal_code'].apply(lambda postal_code: str(postal_code).split('-')[0])

In [494]:
venues['postal_code'].value_counts()

94103    461
94109    409
94110    394
94102    376
94133    335
94117    296
94114    290
94115    270
94123    269
94107    253
94118    223
94122    207
94111    202
94132    189
94105    180
94108    177
94121    176
94112    155
94127    110
94116    100
94129     90
94131     76
94124     70
94104     68
94158     61
94134     54
94130     18
94143      9
94015      8
94014      6
94608      2
94941      2
94199      2
94920      1
94612      1
94113      1
94080      1
94607      1
94578      1
94101      1
94621      1
94619      1
94965      1
94401      1
Name: postal_code, dtype: int64

In [495]:
venues[venues['city'] == 'San Francisco']['postal_code'].value_counts()

94103    461
94109    409
94110    393
94102    375
94133    334
94117    294
94114    288
94123    269
94115    267
94107    253
94118    223
94122    207
94111    202
94132    189
94105    180
94108    177
94121    175
94112    155
94127    110
94116     99
94129     88
94131     76
94124     70
94104     68
94158     60
94134     54
94130     18
94143      9
94014      2
94199      2
94113      1
94101      1
94619      1
94607      1
Name: postal_code, dtype: int64

In [496]:
sf_postal_codes_found = venues[venues['city'] == 'San Francisco']['postal_code'].unique()
sf_postal_codes = sf_geospatial_data_zip['postal_code']
print("There are {} postal codes in San Francisco. But we found {} postal codes.".format(len(sf_postal_codes), len(sf_postal_codes_found)))
sf_err_postal_codes = set(sf_postal_codes_found) - set(sf_postal_codes)
print("Erroroneous postal codes are: {}".format(sf_err_postal_codes))

There are 27 postal codes in San Francisco. But we found 34 postal codes.
Erroroneous postal codes are: {'94113', '94014', '94619', '94199', '94143', '94607', '94101'}


In [505]:
err_postal_codes_records = venues[(venues['postal_code'].isin(sf_err_postal_codes)) & (venues['city'] == 'San Francisco')]
err_postal_codes_records

,address,category,city,country,cross_street,lat,lng,name,postal_code,state
venue_id,,,,,,,,,,
5503632a498ed86c104ab0ef,320 Bowling Green Dr,4bf58dd8d48988d1e7941735,San Francisco,United States,NaN,37.768279,-122.457171,Koret Children's Quarter Playground,94199,CA
4b01caccf964a520754522e3,50 Post St,4bf58dd8d48988d15a941735,San Francisco,United States,at Montgomery St,37.789146,-122.402447,Crocker Galleria Roof Terrace,94101,CA
4b145ba0f964a520c4a123e3,6400 Mission St,4bf58dd8d48988d101951735,San Francisco,United States,Hillside Boulevard,37.705075,-122.462970,One Dollar Only,94014,CA
4ebbf9344fc61a72fca6720d,1675 Owens St,4bf58dd8d48988d137941735,San Francisco,United States,NaN,37.767950,-122.392571,GigaOM Roadmap,94143,CA
5503131b498eefc07489b006,1825 4th St Fl Lbby,4bf58dd8d48988d16d941735,San Francisco,United States,at 16th St,37.766400,-122.390492,Equator Coffee,94143,CA
4c2fc307452620a1c2241e0f,566 Dolores St,4bf58dd8d48988d1d0941735,San Francisco,United States,NaN,37.759981,-122.426831,Truffle Guy,94199,CA
4acbcaa8f964a520b0c720e3,500 Parnassus Ave,4bf58dd8d48988d1b2941735,San Francisco,United States,NaN,37.763754,-122.458410,Millberry Fitness & Recreation Center,94143,CA
569678f3498e45ffc1d9f99a,533 Parnassus Ave,4bf58dd8d48988d1bd941735,San Francisco,United States,NaN,37.762670,-122.458967,Ladle & Leaf,94143,CA
4d6d4a9a423d54814556ecb1,"500 Parnassus Ave,",4bf58dd8d48988d112941735,San Francisco,United States,UCSF Millberry Union,37.763495,-122.458614,Jamba Juice,94619,CA


In [506]:
for index, row in err_postal_codes_records.iterrows():
    lat = row['lat']
    lng = row['lng']
    venues.set_value(index, 'postal_code', get_postal_code(GOOGLE_API_KEY, lat, lng))

In [507]:
venues[venues['city'] == 'San Francisco']['postal_code'].value_counts()

94103    461
94109    409
94110    393
94102    375
94133    335
94117    294
94114    289
94123    269
94115    267
94107    253
94118    223
94122    208
94111    202
94132    189
94105    180
94108    177
94121    176
94112    155
94127    110
94116     99
94129     88
94131     78
94124     70
94104     69
94158     62
94134     54
94130     18
94143      6
94607      1
94014      1
Name: postal_code, dtype: int64

In [513]:
err_city_records = venues[~(venues['postal_code'].isin(sf_postal_codes)) & (venues['city'] == 'San Francisco')]
err_city_records

,address,category,city,country,cross_street,lat,lng,name,postal_code,state
venue_id,,,,,,,,,,
4b145ba0f964a520c4a123e3,6400 Mission St,4bf58dd8d48988d101951735,San Francisco,United States,Hillside Boulevard,37.705075,-122.462970,One Dollar Only,94014,CA
4acbcaa8f964a520b0c720e3,500 Parnassus Ave,4bf58dd8d48988d1b2941735,San Francisco,United States,NaN,37.763754,-122.458410,Millberry Fitness & Recreation Center,94143,CA
4d6d4a9a423d54814556ecb1,"500 Parnassus Ave,",4bf58dd8d48988d112941735,San Francisco,United States,UCSF Millberry Union,37.763495,-122.458614,Jamba Juice,94143,CA
4adcbd4af964a520462f21e3,500 Parnassus Ave,4bf58dd8d48988d1c1941735,San Francisco,United States,btwn Hillway & 3rd,37.763365,-122.458334,Carmelina's Taqueria,94143,CA
47f19991f964a520914e1fe3,500 Parnassus Ave,4bf58dd8d48988d145941735,San Francisco,United States,btwn 3rd Ave & Hillway Ave,37.763365,-122.458364,Panda Express,94143,CA
4cb39781562d224bdbed2e88,"500 Parnassus J Level, Rm Mu-145",4bf58dd8d48988d10f951735,San Francisco,United States,NaN,37.763826,-122.458461,Walgreens,94143,CA
5568b136498ead1a6b55c663,530 Parnassus Ave Kalmanovitz Library,4bf58dd8d48988d1e0931735,San Francisco,United States,3rd Ave,37.763438,-122.459127,Caffé Central,94143,CA
4e569390b0fbb294aaaa60e7,NaN,4bf58dd8d48988d1cb941735,San Francisco,United States,NaN,37.798178,-122.264267,An The Go,94607,CA


In [511]:
google_geocoder = Geocoder(GOOGLE_API_KEY)
result = google_geocoder.reverse_geocode(37.705075, -122.462970)
print(result)
print(result.city)

6400 Mission St, Daly City, CA 94014, USA
Daly City


In [512]:
def get_city(api_key, lat, lng):
    google_geocoder = Geocoder(api_key)
    result = google_geocoder.reverse_geocode(lat, lng)
    return result.city

In [514]:
for index, row in err_city_records.iterrows():
    lat = row['lat']
    lng = row['lng']
    venues.set_value(index, 'city', get_city(GOOGLE_API_KEY, lat, lng))

In [517]:
venues[venues['city'] == 'San Francisco']['postal_code'].value_counts()

94103    461
94109    409
94110    393
94102    375
94133    335
94117    294
94114    289
94123    269
94115    267
94107    253
94118    223
94122    208
94111    202
94132    189
94105    180
94108    177
94121    176
94112    155
94127    110
94116     99
94129     88
94131     78
94124     70
94104     69
94158     62
94134     54
94130     18
94143      6
Name: postal_code, dtype: int64

In [518]:
result = google_geocoder.reverse_geocode(37.763365, -122.458334)
print(result)

500 Parnassus Ave, San Francisco, CA 94143, USA


In [520]:
venues.to_pickle("data/venues.pkl")

In [527]:
def get_category_heirarchy(categories, category_id, stack=None):
    if stack is None:
        stack = []
    
    stack.append(category_id)
    
    if pd.isnull(categories.loc[category_id, 'parent_id']):
        return stack
    
    return get_category_heirarchy(categories, categories.loc[category_id, 'parent_id'], stack)

def get_category_at_level(categories, category_id, level=1):
    levels = get_category_heirarchy(categories, category_id)
    level_id = ''
    for i in range(0, level):
        if (len(levels) > 0):
            level_id = levels.pop()
        else:
            break
    return level_id

def get_category_name(categories, category_id):
    return categories.loc[category_id, 'name']

In [528]:
get_category_heirarchy(categories, "56aa371be4b08b9a8d5734de", stack=None)

['56aa371be4b08b9a8d5734de',
 '4bf58dd8d48988d17f941735',
 '4d4b7104d754a06370d81259']

In [529]:
get_category_name(categories, get_category_at_level(categories, "56aa371be4b08b9a8d5734de", level=2))

'Movie Theater'

In [530]:
venues['category_name'] = venues['category'].apply(lambda cat: get_category_name(categories, cat))
venues['parent_category'] = venues['category'].apply(lambda cat: get_category_at_level(categories, cat, level=2))
venues['parent_category_name'] = venues['parent_category'].apply(lambda cat: get_category_name(categories, cat))

In [531]:
venues.head()

,address,category,city,country,cross_street,lat,lng,name,postal_code,state,category_name,parent_category,parent_category_name
venue_id,,,,,,,,,,,,,
4f3709abe4b0aa62a1d35520,748 Innes Ave,4bf58dd8d48988d1ed941735,San Francisco,United States,NaN,37.730525,-122.372998,Archimedes Banya,94124,CA,Spa,4bf58dd8d48988d1ed941735,Spa
49d3e08bf964a5201f5c1fe3,1634 Jerrold Ave,4bf58dd8d48988d11b951735,San Francisco,United States,NaN,37.739589,-122.390171,Flora Grubb Gardens,94124,CA,Flower Shop,4bf58dd8d48988d11b951735,Flower Shop
53058fdb498e2837542f7f8f,1439 Egbert Ave Unit c,4e0e22f5a56208c4ea9a85a0,San Francisco,United States,3rd Street,37.723004,-122.393615,Seven Stills Brewery & Distillery,94124,CA,Distillery,4e0e22f5a56208c4ea9a85a0,Distillery
555e58ad498e9137bc49f191,900 Marin St,4bf58dd8d48988d1e5931735,San Francisco,United States,btwn Illinois & Michigan St,37.749404,-122.386016,The Midway SF,94124,CA,Music Venue,4bf58dd8d48988d1e5931735,Music Venue
56eb4e0a498e5769545c1843,1525 Cortland Ave,50327c8591d4c4b30a586d5d,San Francisco,United States,NaN,37.739798,-122.408986,Barebottle Brewing Company,94110,CA,Brewery,50327c8591d4c4b30a586d5d,Brewery


In [540]:
venues.to_pickle('data/venues.pkl')

In [9]:
next_venues = pd.read_pickle('data/next_venues.pkl')

In [3]:
next_venues.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19494 entries, 0 to 19493
Data columns (total 2 columns):
venue_id         19494 non-null object
next_venue_id    19494 non-null object
dtypes: object(2)
memory usage: 304.7+ KB


In [4]:
next_venues.head()

,venue_id,next_venue_id
0,4f3709abe4b0aa62a1d35520,43c4e03ff964a5205d2d1fe3
1,49d3e08bf964a5201f5c1fe3,46363302f964a52012461fe3
2,49d3e08bf964a5201f5c1fe3,4f833c3be4b0e533af027908
3,49d3e08bf964a5201f5c1fe3,4bd4afdb29eb9c744a2892e1
4,49d3e08bf964a5201f5c1fe3,4bf313f7370e76b0c956bd4a


In [10]:
next_venues['venue_category'] = next_venues['venue_id'].apply(lambda x: venues.loc[x, 'category'])
next_venues['next_venue_category'] = next_venues['next_venue_id'].apply(lambda x: venues.loc[x, 'category'])

In [14]:
next_venues.head()

,venue_id,next_venue_id,venue_category,next_venue_category
0,4f3709abe4b0aa62a1d35520,43c4e03ff964a5205d2d1fe3,4bf58dd8d48988d1ed941735,50327c8591d4c4b30a586d5d
1,49d3e08bf964a5201f5c1fe3,46363302f964a52012461fe3,4bf58dd8d48988d11b951735,4bf58dd8d48988d1e0931735
2,49d3e08bf964a5201f5c1fe3,4f833c3be4b0e533af027908,4bf58dd8d48988d11b951735,4bf58dd8d48988d1ca941735
3,49d3e08bf964a5201f5c1fe3,4bd4afdb29eb9c744a2892e1,4bf58dd8d48988d11b951735,4eb1c0253b7b52c0e1adc2e9
4,49d3e08bf964a5201f5c1fe3,4bf313f7370e76b0c956bd4a,4bf58dd8d48988d11b951735,4bf58dd8d48988d1c9941735


In [15]:
def get_distance(x):
    return calculate_distance(venues.loc[x['venue_id'], 'lat'], 
                       venues.loc[x['venue_id'], 'lng'], 
                       venues.loc[x['next_venue_id'], 'lat'], 
                       venues.loc[x['next_venue_id'], 'lng'])

def calculate_distance(src_lat, src_lng, dest_lat, dest_lng):
    src = (src_lat, src_lng)
    dest = (dest_lat, dest_lng)
    return distance.distance(src, dest).km

In [16]:
next_venues['distance'] = next_venues.apply(get_distance, axis=1)

In [17]:
next_venues.head()

,venue_id,next_venue_id,venue_category,next_venue_category,distance
0,4f3709abe4b0aa62a1d35520,43c4e03ff964a5205d2d1fe3,4bf58dd8d48988d1ed941735,50327c8591d4c4b30a586d5d,1.122189
1,49d3e08bf964a5201f5c1fe3,46363302f964a52012461fe3,4bf58dd8d48988d11b951735,4bf58dd8d48988d1e0931735,0.003798
2,49d3e08bf964a5201f5c1fe3,4f833c3be4b0e533af027908,4bf58dd8d48988d11b951735,4bf58dd8d48988d1ca941735,0.100066
3,49d3e08bf964a5201f5c1fe3,4bd4afdb29eb9c744a2892e1,4bf58dd8d48988d11b951735,4eb1c0253b7b52c0e1adc2e9,1.506962
4,49d3e08bf964a5201f5c1fe3,4bf313f7370e76b0c956bd4a,4bf58dd8d48988d11b951735,4bf58dd8d48988d1c9941735,2.038199


In [18]:
next_venues.to_pickle("data/next_venues.pkl")